# Synthetic Data Generation Tutorial using Translation Pipeline

This tutorial demonstrates how to use SDG repository to generate synthetic question-answer pairs from documents using large language models like LLaMA 3.3 70B. We will also generate data using Mixtral model for comparison. We'll cover:

1. Setting up the environment
2. Connecting to LLM servers
3. Configuring the data generation pipeline
4. Generating data with different models
5. Comparing results

In [ ]:
# Enable auto-reloading of modules - useful during development
%load_ext autoreload
%autoreload 2

### Setup Instructions

Before running this notebook, you'll need to:

```bash 
pip install sdg-hub==0.1.0a2
```

In [1]:
# Import required libraries
# datasets: For handling our data
# OpenAI: For interfacing with the LLM servers
# SDG components: For building our data generation pipeline
from datasets import load_dataset, Dataset
from openai import OpenAI

from sdg_hub.flow import Flow
from sdg_hub.pipeline import Pipeline
from sdg_hub.sdg import SDG
from sdg_hub.registry import PromptRegistry

/Users/rudramurthy/miniforge3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Setting up LLaMA 3.3 70B Model

First, we need to host the LLaMA model using vLLM. This creates an OpenAI-compatible API endpoint.

1. Start the vLLM server (run in terminal):
```bash
CUDA_VISIBLE_DEVICES=0,1,2,3,4,5,6,7 python -m vllm.entrypoints.openai.api_server \
    --model meta-llama/Llama-3.3-70B-Instruct \
    --dtype float16 \
    --tensor-parallel-size 8 
```

2. Connect to the model using OpenAI client below:

In [2]:
# Configure OpenAI client to connect to our local vLLM server
endpoint = f"http://localhost:8000/v1"
openai_api_key = "EMPTY"  # vLLM doesn't require real API key
openai_api_base = endpoint

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

# Verify we can see the model
teacher_model = client.models.list().data[0].id
print(f"Connected to model: {teacher_model}")

[21:38:32] INFO     HTTP Request: GET http://localhost:8000/v1/models "HTTP/1.1 200 OK"             ]8;id=979993;file:///Users/rudramurthy/miniforge3/envs/py310/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=328405;file:///Users/rudramurthy/miniforge3/envs/py310/lib/python3.10/site-packages/httpx/_client.py#1025\1025]8;;\

Connected to model: Qwen/Qwen2.5-1.5B-Instruct


### Configure Qwen 2.5 1B Prompt Template

We need to register the correct chat template for our model to ensure proper prompt formatting.

In [3]:
# Register the LLaMA 3.3 chat template
# This ensures proper formatting of prompts for the model
from transformers import AutoTokenizer

# Load the tokenizer to get the chat template
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")

# Register the chat template in our prompt registry
@PromptRegistry.register("Qwen/Qwen2.5-1.5B-Instruct")
def llama_3_3_70b_chat_template():
    return tokenizer.chat_template

### Configure the Data Generation Pipeline

Now we'll set up our Synthetic Data Generation (SDG) pipeline with the following components:
1. SDG Flow configuration from YAML
2. SDG Pipeline setup
3. SDG configuration with batch processing, number of workers, and save frequency parameters

In [13]:
knowledge_agentic_pipeline = "/Users/rudramurthy/Documents/GitHub/new_ilab_sdg/src/sdg_hub/flows/generation/knowledge/translate_knowledge.yaml"
flow_cfg = Flow(client).get_flow_from_file(knowledge_agentic_pipeline)
sdg = SDG(
    [Pipeline(flow_cfg)],
    num_workers=1,
    batch_size=1,
    save_freq=1000,
)

### Load and Prepare Seed Data

We'll load our seed data (documents) that will be used to generate question-answer pairs.

In [5]:
# Load the seed data from JSON file
number_of_samples = 3
seed_data_dir = f"sdg_demo_output/"
ds = load_dataset('json', data_files=f'{seed_data_dir}/seed_data.jsonl', split='train')
ds = ds.shuffle(seed=42).select(range(number_of_samples))

### Generate Data with Qwen 2.5

Now we'll use our configured pipeline to generate synthetic question-answer pairs.

In [ ]:
# Generate synthetic data and save checkpoints
generated_data = sdg.generate(ds, checkpoint_dir="Tmp")

[21:44:20] INFO     No existing checkpoints found in Tmp, generating from scratch                        ]8;id=605874;file:///Users/rudramurthy/Documents/GitHub/new_ilab_sdg/src/sdg_hub/sdg.py\sdg.py]8;;\:]8;id=83270;file:///Users/rudramurthy/Documents/GitHub/new_ilab_sdg/src/sdg_hub/sdg.py#107\107]8;;\

           INFO     Splitting the dataset into smaller batches                                           ]8;id=568676;file:///Users/rudramurthy/Documents/GitHub/new_ilab_sdg/src/sdg_hub/sdg.py\sdg.py]8;;\:]8;id=462980;file:///Users/rudramurthy/Documents/GitHub/new_ilab_sdg/src/sdg_hub/sdg.py#123\123]8;;\

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 38362.54it/s]


           INFO     Generating dataset with 3 splits, batch size 1, and 1 workers                        ]8;id=2433;file:///Users/rudramurthy/Documents/GitHub/new_ilab_sdg/src/sdg_hub/sdg.py\sdg.py]8;;\:]8;id=334455;file:///Users/rudramurthy/Documents/GitHub/new_ilab_sdg/src/sdg_hub/sdg.py#129\129]8;;\

           INFO     Processing split 0                                                                    ]8;id=340068;file:///Users/rudramurthy/Documents/GitHub/new_ilab_sdg/src/sdg_hub/sdg.py\sdg.py]8;;\:]8;id=429099;file:///Users/rudramurthy/Documents/GitHub/new_ilab_sdg/src/sdg_hub/sdg.py#75\75]8;;\

...

           ERROR    Error processing split 0: Pipeline stopped: Empty dataset after running block:        ]8;id=995813;file:///Users/rudramurthy/Documents/GitHub/new_ilab_sdg/src/sdg_hub/sdg.py\sdg.py]8;;\:]8;id=416319;file:///Users/rudramurthy/Documents/GitHub/new_ilab_sdg/src/sdg_hub/sdg.py#82\82]8;;\
                    question_response_generation                                                                   

Traceback (most recent call last):
  File "/Users/rudramurthy/Documents/GitHub/new_ilab_sdg/src/sdg_hub/sdg.py", line 79, in _generate_data
    input_split = pipeline.generate(input_split)
  File "/Users/rudramurthy/Documents/GitHub/new_ilab_sdg/src/sdg_hub/pipeline.py", line 52, in generate
    raise EmptyDatasetError(
sdg_hub.pipeline.EmptyDatasetError: Pipeline stopped: Empty dataset after running block: question_response_generation
 33%|███████████████████████████████████████████████████                                                                                                      | 1/3 [03:19<06:38, 199.28s/it]

           INFO     Processing split 1                                                                    ]8;id=905130;file:///Users/rudramurthy/Documents/GitHub/new_ilab_sdg/src/sdg_hub/sdg.py\sdg.py]8;;\:]8;id=263254;file:///Users/rudramurthy/Documents/GitHub/new_ilab_sdg/src/sdg_hub/sdg.py#75\75]8;;\

           WARNING  Loading facebook/nllb-200-distilled-600M model...                        ]8;id=676853;file:///Users/rudramurthy/Documents/GitHub/new_ilab_sdg/src/sdg_hub/blocks/translationblock.py\translationblock.py]8;;\:]8;id=350391;file:///Users/rudramurthy/Documents/GitHub/new_ilab_sdg/src/sdg_hub/blocks/translationblock.py#76\76]8;;\


document_translation Prompt Generation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:15<00:00, 15.78s/it]


### Run SDG through python command (For large scale generation)

```python
python /home/lab/sdg/scripts/generate.py --ds_path {output_dir}/seed_data.jsonl --bs 8 --num_workers 8 --save_path {output_dir}/gen.jsonl --flow ../src/instructlab/sdg/flows/generation/knowledge/synth_knowledge1.5.yaml --endpoint {teacher_endpoint_url} --checkpoint_dir {output_dir}/data_checkpoints --save_freq 2
```

### Save the generated data into training format

In [ ]:
from sdg_hub.utils.parse_and_convert import create_knowledge_regular_ds, create_knowledge_pretraining_ds
from datasets import concatenate_datasets

output_dir = f"sdg_demo_output/"

# Add the system prompt to final dataset if needed. For 
#  we use system prompt similar to below
system_prompt_lab = (
    "I am a LAB Instruct Model, an AI language model developed by Red Hat and IBM Research based on the granite-3.1-8b-base model. My primary role is to serve as a chat assistant."
)

# This is a general instruction tuning dataset that is mixed with generated knowledge to train LLM simultaneously on your knowledge and general instructions.
precomputed_skills_path = "<LAB precomputed skills path>"
precomputed_skills = load_dataset('json', data_files=precomputed_skills_path, split='train')

generated_ds = load_dataset('json', data_files=f'{output_dir}/gen.jsonl', split='train')

# Create Pretraining Knowledge Dataset (Also known as Phase 0.7/Phase 7)
phase_0_7_ds = create_knowledge_pretraining_ds(generated_ds)
phase_0_7_ds.to_json(f'{output_dir}/phase_0_7_ds.jsonl', orient='records', lines=True)

# Create Regular Knowledge Dataset (Also known as Phase 1.0/Phase 10)
phase_1_ds = create_knowledge_regular_ds(generated_ds)

# Mix the pre-computed skills with the regular knowledge dataset. If more than one dataset were generated simply add those in this concatenation stage.
# If you have any generated instruction data, that can be also mixed in this stage. If you only have generated skills phase 07 generation and training can be skipped.
phase_1_ds = concatenate_datasets([phase_1_ds, precomputed_skills])
phase_1_ds.to_json(f'{output_dir}/phase_1_ds.jsonl', orient='records', lines=True)